In [2]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.nn import functional as F

import onnx
import onnxruntime.quantization
import onnxruntime
from onnxruntime.quantization import quantize_qat, quantize_static, QuantType

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
!nvidia-smi

Sun Jul  4 03:45:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.30       Driver Version: 462.30       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 2060   WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   51C    P8     8W /  N/A |   1385MiB /  6144MiB |     17%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# WANDB LIBRARY
### IMPORT WANDB AND LOGIN
Note you may have to login using your API key


In [5]:
import wandb
%env "WANDB_NOTEBOOK_NAME" "demo_wine_wandb_test"
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


env: "WANDB_NOTEBOOK_NAME"="demo_wine_wandb_test"


wandb: Currently logged in as: markgich (use `wandb login --relogin` to force relogin)


True

### Import Dataset

In [6]:
df = pd.read_csv("./data/wine_data.csv")

In [7]:
df.head()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,0,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,0,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,0,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,0,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [8]:
df.shape

(178, 14)

In [9]:
df.describe()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,0.938202,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.775035,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,0.000000,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,0.000000,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,1.000000,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,2.000000,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,2.000000,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


# DATA WRANGLING
### Check for Nulls and Duplicates

In [10]:
df.isna().sum()

Class                           0
Alcohol                         0
Malic acid                      0
Ash                             0
Alcalinity of ash               0
Magnesium                       0
Total phenols                   0
Flavanoids                      0
Nonflavanoid phenols            0
Proanthocyanins                 0
Color intensity                 0
Hue                             0
OD280/OD315 of diluted wines    0
Proline                         0
dtype: int64

In [11]:
df.duplicated().sum()

0

# MACHINE LEARNING
### ML PREP

In [12]:
# Encode target labels with value between 0 and n_classes-1.
# Import Metrics for use with evaluation

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report


In [13]:
le = LabelEncoder()
df['Class'] = le.fit_transform(df['Class'])
df.sample(10)

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
37,0,13.05,1.65,2.55,18.0,98,2.45,2.43,0.29,1.44,4.25,1.12,2.51,1105
59,1,12.37,0.94,1.36,10.6,88,1.98,0.57,0.28,0.42,1.95,1.05,1.82,520
72,1,13.49,1.66,2.24,24.0,87,1.88,1.84,0.27,1.03,3.74,0.98,2.78,472
5,0,14.20,1.76,2.45,15.2,112,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450
114,1,12.08,1.39,2.50,22.5,84,2.56,2.29,0.43,1.04,2.90,0.93,3.19,385
73,1,12.99,1.67,2.60,30.0,139,3.30,2.89,0.21,1.96,3.35,1.31,3.50,985
13,0,14.75,1.73,2.39,11.4,91,3.10,3.69,0.43,2.81,5.40,1.25,2.73,1150
142,2,13.52,3.17,2.72,23.5,97,1.55,0.52,0.50,0.55,4.35,0.89,2.06,520
48,0,14.10,2.02,2.40,18.8,103,2.75,2.92,0.32,2.38,6.20,1.07,2.75,1060
69,1,12.21,1.19,1.75,16.8,151,1.85,1.28,0.14,2.50,2.85,1.28,3.07,718


In [14]:
df['Class'].unique()

array([0, 1, 2], dtype=int64)

### SEPARATE FEATURES AND TARGET

In [15]:
# set the feature variables

df_features = df.drop('Class', axis=1)
df_features.head()

,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [16]:
# Set the target variable

df_target = df[['Class']]
df_target.head()

,Class
0,0
1,0
2,0
3,0
4,0


In [17]:
# Split dataset in train and test with a ratio of 70-30

from sklearn.model_selection import train_test_split

In [18]:
X_train, x_test, Y_train, y_test = train_test_split(df_features, 
                                                    df_target,
                                                    test_size=0.3,
                                                     random_state=42)

In [19]:
X_train.shape, x_test.shape,

((124, 13), (54, 13))

In [20]:
 Y_train.shape, y_test.shape

((124, 1), (54, 1))

### Convert data to Tensors for Pytorch

In [21]:
Xtrain = torch.from_numpy(X_train.values).float()
Xtest = torch.from_numpy(x_test.values).float()
print(Xtrain.shape, Xtest.shape)

torch.Size([124, 13]) torch.Size([54, 13])


In [22]:
print(Xtrain.dtype, Xtest.dtype)

torch.float32 torch.float32


We have successfully converted our  X_data into torch tensors of float32 datatype

In [23]:
# Reshape tensor to 1D

Ytrain = torch.from_numpy(Y_train.values).view(1,-1)[0]
Ytest = torch.from_numpy(y_test.values).view(1, -1)[0]
print(Ytrain.shape, Ytest.shape)

torch.Size([124]) torch.Size([54])


We use the **view()** to reshape the tensor.<br>
The loss function doesn't support multi-target and therefore, we should use a 1D Tensor of 1 row containing the labels.<br>
We have successfully converted our y_data

In [24]:
print(Ytrain.dtype, Ytest.dtype)

torch.int64 torch.int64


## PyTorch
### We create a classifier and define our neural network for our model

### Hyperparameters

In [25]:
input_size = 13
output_size = 3
hidden_size = 100

In [26]:
config = dict(
                input_size = 13,
                output_size = 3,
                hidden_size = 100,
                dataset = "wine dataset",
                architecture = 'Linear', 
                learning_rate = 0.01,
                loss = nn.NLLLoss(),
                optimizer = "adam",
)


In [27]:
for k,v in config.items():
    print(k, v)

input_size 13
output_size 3
hidden_size 100
dataset wine dataset
architecture Linear
learning_rate 0.01
loss NLLLoss()


### Define the neural network


In [28]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, X):
        X = torch.sigmoid((self.fc1(X)))
        X = torch.sigmoid(self.fc2(X))
        X = self.fc3(X)

        return F.log_softmax(X, dim=-1)

In [29]:
model = Net()
# preview out model
model

Net(
  (fc1): Linear(in_features=13, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=3, bias=True)
)

### Define Optimizer and Loss Function

In [30]:
import torch.optim as optim

In [31]:
optimizer = optim.Adam(model.parameters(), lr=config.get("learning_rate"))
loss_fn = config.get("loss")

In [ ]:
  # Define the optimizer
    if config.optimizer=='sgd':
      optimizer = optim.SGD(model.parameters(),lr=config.get("learning_rate"), momentum=0.9)
    elif config.optimizer=='adam':
      optimizer = optim.Adam(model.parameters(),lr=config.get("learning_rate"))

# TRAIN AND LOG THE MODEL
### Train the Model for 1000 Epochs
### Log the model Parameters
### Export Model, Import and Set to Eval
### Log Metrics on Model.Eval
### Convert and Export to ONNX

In [32]:
# TRAIN THE MODEL

epochs = 1000
with wandb.init(project="demo_wandb_test", config = config):
    wandb.watch(model, criterion=None, log="gradients", log_freq=10)
    
    for epoch in range(epochs):
        
        optimizer.zero_grad()
        Ypred = model(Xtrain)

        loss = loss_fn(Ypred, Ytrain)
        loss.backward()

        optimizer.step()

        wandb.log({'Epoch': epoch, "Loss": loss.item()})


    # SAVE MODEL STATE DICT TO DISK

    wandb.save(torch.save(model.state_dict(), "./models/home_state_dict.pt"))

    # LOAD MODEL FROM DISK and EVALUATE

    new_model =  Net()
    new_model.load_state_dict(torch.load("./models/home_state_dict.pt"))
    new_model.eval()

    # SET THE PREDICTIONS

    predict = new_model(Xtest)
    _, predict_y = torch.max(predict, 1)

    # VISUALIZE CONFUSION MATRIX

    wandb.sklearn.plot_confusion_matrix(Ytest, predict_y, labels = [0,1,2])
    # Print Metrics

    wandb.log({"accuracy_score" : accuracy_score(Ytest, predict_y),
    "precision_score" : precision_score(Ytest, predict_y, average='weighted'),
    "recall_score": recall_score(Ytest, predict_y, average="weighted")})
    
    torch.onnx.export(model = model,args =  (Xtrain), f = "./models/home_state_test.onnx", input_names=['input'], output_names = ['output'],
    verbose=True, do_constant_folding=True, opset_version=11)
wandb.finish()


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


graph(%input : Float(124, 13, strides=[1, 124], requires_grad=0, device=cpu),
      %fc1.weight : Float(100, 13, strides=[13, 1], requires_grad=1, device=cpu),
      %fc1.bias : Float(100, strides=[1], requires_grad=1, device=cpu),
      %fc2.weight : Float(100, 100, strides=[100, 1], requires_grad=1, device=cpu),
      %fc2.bias : Float(100, strides=[1], requires_grad=1, device=cpu),
      %fc3.weight : Float(3, 100, strides=[100, 1], requires_grad=1, device=cpu),
      %fc3.bias : Float(3, strides=[1], requires_grad=1, device=cpu)):
  %7 : Float(124, 100, strides=[100, 1], requires_grad=1, device=cpu) = onnx::Gemm[alpha=1., beta=1., transB=1](%input, %fc1.weight, %fc1.bias) # c:\Users\markg\venv\torch-venv\lib\site-packages\torch\nn\functional.py:1753:0
  %8 : Float(124, 100, strides=[100, 1], requires_grad=1, device=cpu) = onnx::Sigmoid(%7) # <ipython-input-28-c3aa4c118253>:10:0
  %9 : Float(124, 100, strides=[100, 1], requires_grad=1, device=cpu) = onnx::Gemm[alpha=1., beta=1., tra

Epoch,999
Loss,0.00046
_runtime,11
_timestamp,1625359550
_step,1001
accuracy_score,0.88889
precision_score,0.90063
recall_score,0.88889


Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Loss,█▅▄▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▃▃▃▃▆▆▆▆▆▆▆▆▆▆▆███████
_timestamp,▁▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▃▃▃▃▆▆▆▆▆▆▆▆▆▆▆███████
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
accuracy_score,▁
precision_score,▁
recall_score,▁


# OPTIMIZE NETWORK WITH SWEEPS
### WANDB SWEEPS

In [33]:
# Replace agent with own agent line generated from project
!wandb agent markgich/demo_wandb_test/izsvluxh

wandb: Starting wandb agent 🕵️
2021-07-04 04:04:30,929 - wandb.wandb_agent - INFO - Running runs: []
2021-07-04 04:04:31,367 - wandb.wandb_agent - INFO - Agent received command: run
2021-07-04 04:04:31,368 - wandb.wandb_agent - INFO - Agent starting run with config:
	architecture: Linear
	dataset: wine dataset
	hidden_size: 186
	input_size: 20
	learning_rate: 0.018638395920396605
	loss: NLLLoss()
	output_size: 6
2021-07-04 04:04:31,370 - wandb.wandb_agent - INFO - About to run command: python train.py --architecture=Linear "--dataset=wine dataset" --hidden_size=186 --input_size=20 --learning_rate=0.018638395920396605 --loss=NLLLoss() --output_size=6
python: can't open file 'train.py': [Errno 2] No such file or directory
2021-07-04 04:04:36,383 - wandb.wandb_agent - INFO - Running runs: ['2pn2ac2m']
2021-07-04 04:04:36,383 - wandb.wandb_agent - INFO - Cleaning up finished run: 2pn2ac2m
2021-07-04 04:04:36,838 - wandb.wandb_agent - INFO - Agent received command: run
2021-07-04 04:04:36,8

In [ ]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'epochs': {
            'values': [100, 500, 1000]
        },
        
        'learning_rate': {
            'values': [1e-2, 1e-3, 1e-4, 3e-4, 3e-5, 1e-5]
        },
        'fc_layer_size':{
            'values':[128,256,512]
        },
        'optimizer': {
            'values': ['adam', 'sgd']
        },
    }
}